In [89]:
from farmhash import FarmHash32 as fh
import pandas as pd
import pytz
import boto3
import json
import os
import datetime
from json.decoder import JSONDecodeError
from botocore.exceptions import ClientError
import logging
import re

from src import utils

In [90]:
file_path = '/home/team3/repository/keys/'

In [91]:
os.listdir(file_path)
with open(f'{file_path}/API_KEYS.json', 'r') as f:
    key = json.load(f)
with open(f'{file_path}/DATA_SRC_INFO.json', 'r') as f:
    bucket_info = json.load(f)

In [92]:
pull_bucket_name = bucket_info['pull_bucket_name']
restore_table_name = bucket_info['restore_table_name']
target_folder_prefix = bucket_info['target_folder_prefix']['rocketpunch_path']

In [93]:
# S3 섹션 및 client 생성
session = boto3.Session(
    aws_access_key_id=key['aws_access_key_id'],
    aws_secret_access_key=key['aws_secret_key'],
    region_name=key['region']
)

s3 = session.client('s3')

In [94]:
# 특정 폴더 내 파일 목록 가져오기
# TODO: 
# - 마지막 실행일(년,월,일)을 datetime으로 저장한 파일을 읽어들여 curr_date에 적용하기; 당담: 유정연
response = s3.list_objects_v2(Bucket=pull_bucket_name, Prefix=target_folder_prefix, Delimiter='/')
curr_date = datetime.datetime.now(pytz.timezone('Asia/Seoul')).date()  # 로컬 시간대(UTC+9)로 현재 날짜 설정
kst_tz = pytz.timezone('Asia/Seoul') # kst timezone 설정
#curr_date = datetime.date(2024, 8, 21)

# curr_date 보다 날짜가 늦은 data josn 파일 metadata 객체 분류
if 'Contents' in response:
    target_file_list = [obj for obj in response['Contents'] if curr_date <= obj['LastModified'].astimezone(kst_tz).date()]
else:
    print("No objects found in the folder.")

In [95]:
for obj in target_file_list:
    try:
        response = s3.get_object(Bucket=pull_bucket_name, Key=obj['Key'])
        json_context = response['Body'].read().decode('utf-8')
        cleaned_text = re.sub(r'[\r\u2028\u2029]+', ' ', json_context) # 파싱을 위해 unuseal line terminators 제거
        json_list = [json.loads(line) for line in cleaned_text.strip().splitlines()] # pandas format으로 맞추기
        print(json_list)
        df = pd.DataFrame(json_list)
    except JSONDecodeError as e:
        logging.error(f"JSONDecodeError encountered: {e}")
        continue
    except ClientError as e:
        logging.error(f"ClientError encountered: {e}")
        continue
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        continue
    
    break

In [96]:
df.head(2)

,company_id,company_name,description,job_id,job_title,job_career,timestamp,crawl_domain,job_url,date_end,date_start,job_task,job_specialties,job_detail,job_industry
0,139154,크레이빙콜렉터,"누구나 낭비 없이 재미있게 패션을 즐길 수 있도록, 세컨핸드 패션 거래 플랫폼 '콜...",151339,HR 매니저,[경력],2024-08-22_17:16:43,www.rocketpunch.com,https://www.rocketpunch.com/jobs/151339,None,2024-08-22,- 크레이빙콜렉터의 인사 관리 담당- 풀스택 HR 업무- 팀 구성 및 기술 스택 관리,"인사, 채용, 채용(HR)",자격 요건- 기업 경력에서 2년 이상의 HR 매니저 경험- 인사 플랫폼을 통한 후보...,"플랫폼, 패션, 커머스, 앱, 어플리케이션, e-커머스, 앱/웹개발, 패션&라이프스..."
1,139154,크레이빙콜렉터,"누구나 낭비 없이 재미있게 패션을 즐길 수 있도록, 세컨핸드 패션 거래 플랫폼 '콜...",151647,그래픽 디자이너(프리랜서),"[신입, 경력]",2024-08-22_17:16:43,www.rocketpunch.com,https://www.rocketpunch.com/jobs/151647,None,2024-08-22,"- 앱 내 배너 & 상세 페이지, 온드미디어 채널(인스타그램, 블로그, 포스트 등)...","그래픽 디자인, 브랜드 디자인, Adobe Photoshop, Adobe Illus...",근무 형태- 원격 근무채용 정보- 채용 직무 : 그래픽 디자이너(프리랜서)- 소속 ...,"플랫폼, 패션, 커머스, 앱, 어플리케이션, e-커머스, 앱/웹개발, 패션&라이프스..."


In [9]:
processing_df = pd.DataFrame()

In [72]:
df = pd.read_json('./rocketpunch_20240822.json')

In [75]:
df.head(2)

,company_id,company_name,description,job_id,job_title,job_career,timestamp,crawl_domain,job_url,date_end,date_start,job_task,job_specialties,job_detail,job_industry
0,139154,크레이빙콜렉터,"누구나 낭비 없이 재미있게 패션을 즐길 수 있도록, 세컨핸드 패션 거래 플랫폼 '콜...",151339,HR 매니저,[경력],2024-08-22_17:16:43,www.rocketpunch.com,https://www.rocketpunch.com/jobs/151339,None,2024-08-22,- 크레이빙콜렉터의 인사 관리 담당- 풀스택 HR 업무- 팀 구성 및 기술 스택 관리,"인사, 채용, 채용(HR)",자격 요건- 기업 경력에서 2년 이상의 HR 매니저 경험- 인사 플랫폼을 통한 후보...,"플랫폼, 패션, 커머스, 앱, 어플리케이션, e-커머스, 앱/웹개발, 패션&라이프스..."
1,139154,크레이빙콜렉터,"누구나 낭비 없이 재미있게 패션을 즐길 수 있도록, 세컨핸드 패션 거래 플랫폼 '콜...",151647,그래픽 디자이너(프리랜서),"[신입, 경력]",2024-08-22_17:16:43,www.rocketpunch.com,https://www.rocketpunch.com/jobs/151647,None,2024-08-22,"- 앱 내 배너 & 상세 페이지, 온드미디어 채널(인스타그램, 블로그, 포스트 등)...","그래픽 디자인, 브랜드 디자인, Adobe Photoshop, Adobe Illus...",근무 형태- 원격 근무채용 정보- 채용 직무 : 그래픽 디자이너(프리랜서)- 소속 ...,"플랫폼, 패션, 커머스, 앱, 어플리케이션, e-커머스, 앱/웹개발, 패션&라이프스..."


In [100]:
for i, data in df.iterrows() :
    # rocketpunch key > merged key
    # job_task > job_tasks
    if pd.notnull(data['job_task']) :
        processing_df.at[i, 'job_tasks'] = ' '.join(
            [item for item in re.sub(r'[^.,/\-+()\s\w]', ' ', \
                re.sub(r'\\/', '/', data['job_task'])).split() if item not in ['-', '+']]
        )
    # job_specialites > stacks
    processing_df['stacks'] = re.sub(r'\\/', '/', data['job_specialties'])
    # job_detail > job_requirements
    processing_df['job_requirements'] = ' '.join(
        [item for item in re.sub(r'[^.,/\-+()\s\w]', ' ', \
            re.sub(r'\\/', '/', data['job_detail'])).split() if item not in ['-', '+']]
        )
    # job_industry > indurstry_type
    processing_df['indurstry_type'] = re.sub(r'\\/', '/', data['job_industry'])
    # date_start > start_date
    date_start = datetime.datetime.strptime(data['date_start'], "%Y-%m-%d")
    processing_df['start_date'] = date_start.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    # date_end > end_date
    if data['date_end']:
        date_end = datetime.datetime.strptime(data['date_end'], "%Y-%m-%d")
        processing_df['end_date'] = date_end.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    
    #job_career > required_career
    processing_df['required_career'] = any(career in "신입" for career in data['job_career'])
    
    # symbol 추가
    processing_df['symbol'] = "RP"
    
    # id
    processing_df['id'] = fh("RP" + data['company_name'] + str(data['job_id']))
    
    
    
    
processing_df.head(4)

,job_tasks,stacks,job_requirements,indurstry_type,required_career,symbol,start_date,end_date,id
0,크레이빙콜렉터의 인사 관리 담당- 풀스택 HR 업무- 팀 구성 및 기술 스택 관리,"powerpoint, Figma, Exel, MS Word",담당업무- 신규 서비스 기획(운영/관리/리딩) 총괄- 전반적인 프로젝트 리딩자격요건...,SW,False,RP,2024-04-24 00:00:00.000,2024-09-30 00:00:00.000,3127825520
1,"앱 내 배너 상세 페이지, 온드미디어 채널(인스타그램, 블로그, 포스트 등) 콘텐츠...","powerpoint, Figma, Exel, MS Word",담당업무- 신규 서비스 기획(운영/관리/리딩) 총괄- 전반적인 프로젝트 리딩자격요건...,SW,False,RP,2024-04-24 00:00:00.000,2024-09-30 00:00:00.000,3127825520
2,사업/파트너십 기획서 작성- 콜렉티브 셀러 영입 및 관리- 다양한 파트너들과의 협업...,"powerpoint, Figma, Exel, MS Word",담당업무- 신규 서비스 기획(운영/관리/리딩) 총괄- 전반적인 프로젝트 리딩자격요건...,SW,False,RP,2024-04-24 00:00:00.000,2024-09-30 00:00:00.000,3127825520
3,콜렉티브 앱 및 웹 개발 전담- 풀스택 개발- 팀 구성 및 기술 스택 결정,"powerpoint, Figma, Exel, MS Word",담당업무- 신규 서비스 기획(운영/관리/리딩) 총괄- 전반적인 프로젝트 리딩자격요건...,SW,False,RP,2024-04-24 00:00:00.000,2024-09-30 00:00:00.000,3127825520


In [101]:
processing_df.columns.__len__()

9

In [102]:
processing_df.__len__

<bound method DataFrame.__len__ of                                              job_tasks  \
0        크레이빙콜렉터의 인사 관리 담당- 풀스택 HR 업무- 팀 구성 및 기술 스택 관리   
1    앱 내 배너 상세 페이지, 온드미디어 채널(인스타그램, 블로그, 포스트 등) 콘텐츠...   
2    사업/파트너십 기획서 작성- 콜렉티브 셀러 영입 및 관리- 다양한 파트너들과의 협업...   
3            콜렉티브 앱 및 웹 개발 전담- 풀스택 개발- 팀 구성 및 기술 스택 결정   
4    주요 업무- 앱 내 배너 상세 페이지, 온드미디어 채널(인스타그램, 블로그, 포스트...   
..                                                 ...   
593  네이티브 Android, IOS 모바일 클라이언트 개발(리뉴얼)AOS(안드로이드 스...   
594  서비스 내 활용되는 의류 모델링 작업- 유니티(Unity)에서 Cloth 옵션 적용...   
595  word embedding, text clustering 등 NLP 기술을 이용한 ...   
596  1. 직무 설명- 멀티플랫폼 어플리케이션 및 홈페이지 개발- 복잡한 워크플로우 및 ...   
597  GenomeFi는 AI 기반 Web3 Genome DID 플랫폼을 통해 게놈, 유전...   

                               stacks  \
0    powerpoint, Figma, Exel, MS Word   
1    powerpoint, Figma, Exel, MS Word   
2    powerpoint, Figma, Exel, MS Word   
3    powerpoint, Figma, Exel, MS Word   
4    powerpoint, Figma, Exel, MS Word   
..                    